<a href="https://colab.research.google.com/github/hindesh-akash/sebi-production-project/blob/main/youtube_details_extraction_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
from IPython.display import clear_output

!pip install pytube
!pip install openai
!pip install google-api-python-client


clear_output(wait=False)

In [72]:

import openai
from pytube import YouTube
import requests
from PIL import Image
import os
import time

openai.api_key = "------OPEN_AI API KEY"


In [59]:
def transcribe_audio(audio_file):

    """
    Function to transcribe an audio file

            Parameters:
                    audio_file (bytes): The audio file in byte like format
            Returns:
                    transcribed_text (string): The transcription of the audio file in text format
    """
    transcript = openai.Audio.transcribe("whisper-1", audio_file)
    transcribed_text = transcript['text']
    return transcribed_text

In [60]:
# def get_text_from_image(image_file):
#     #Convert image to text
#     # img = Image.open(image_file)
#     text = pytesseract.image_to_string(img)

#     return text

In [61]:
def get_thumbnail(video_link):
  """
    Function to get thumbnail of a YouTube video

            Parameters:
                    video_link (string): Hyperlink for a YouTube video.
            Returns:
                    image: The thumbnail image of the video.
  """
  utube = YouTube(video_link)
  thumbnail_url = utube.thumbnail_url
  thumbnail_response = requests.get(thumbnail_url)

  with open('i.png','wb') as f:
    f.write(thumbnail_response.content)

  image = Image.open('i.png')
  os.remove('i.png')

  return image

In [62]:
def get_details(video_link):

    """
    Function to return details about a YouTube video given its hyperlink

          Parameters:
                  video_link (string): Hyperlink for a YouTube video.
                                       Example- https://www.youtube.com/shorts/VIDEO_ID or https://www.youtube.com/watch?v=VIDEO_ID

          Returns:
                  video_id (string): Unique identifier for the video. Example- YDc_wyAjHME
                  video_title (string): Title of the YouTube video
                  video_description (string): Description of the video
                  channel_name (string): Name of the YouTube channel
                  video_num_views (string): Number of views on the video
                  video_num_likes (string): Number of likes on the video
                  video_num_comments (string): Number of comments on the video
                  channel_num_subscribers (string): Number of subsribers for the YouTube channel
                  transcript (string): The transcript of the video



    """

    if video_link.find("shorts"):
      video_id = video_link.split("/")[-1]
    else:
      video_id = video_link.split("=")[1]


    #---------------------------------------
    request = youtube.videos().list(
    part=["statistics","snippet"],
    id=video_id
    )
    response = request.execute()

    video_title = response['items'][0]['snippet']['title']
    video_description = response['items'][0]['snippet']['description']
    channel_id = response['items'][0]['snippet']['channelId']
    video_num_views = response['items'][0]['statistics']['viewCount']
    video_num_likes = response['items'][0]['statistics']['likeCount']
    video_num_comments = response['items'][0]['statistics']['commentCount']

    #---------------------------------------
    request2 = youtube.channels().list(
    part=["snippet","statistics"],
    id=channel_id
    )
    response2 = request2.execute()

    channel_name = response2['items'][0]['snippet']['title']
    chanel_description = response2['items'][0]['snippet']['description']
    channel_num_subscribers = response2['items'][0]['statistics']['subscriberCount']
    channel_num_videos = response2['items'][0]['statistics']['videoCount']

    #---------------------------------------
    request3 = youtube.commentThreads().list(
    part="snippet",
    videoId=video_id,
    maxResults=10
    )

    response3 = request3.execute()
    video_comments = [item['snippet']['topLevelComment']['snippet']['textOriginal'] for item in response3['items']]

    #---------------EXTRACTING TRANSCRIPT----------------
    # Initialize a YouTube object
    yt = YouTube(video_link)
    audio_stream = yt.streams.filter(only_audio=True, file_extension="webm").last()
    audio_stream.download(filename='audio_temp.wav')
    audio_file =  open('audio_temp.wav', "rb")

    time.sleep(10)
    transcript = transcribe_audio(audio_file)
    audio_file.close()
    os.remove('audio_temp.wav')


    return [video_id, video_title, video_description, channel_name, video_num_views, video_num_likes,
            video_num_comments, channel_num_subscribers, transcript]

In [63]:
url = 'https://www.youtube.com/shorts/YDc_wyAjHME'

In [64]:
# video_details = get_details(url)

In [65]:
# video_details

In [ ]:
from IPython.display import clear_output

!pip install fastapi
!pip install langchain
!pip install dotenv
!pip install youtube-transcript-api

# clear_output()

In [81]:
import json
from pytube import extract
from fastapi import APIRouter

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage

from pytube import YouTube
from youtube_transcript_api import YouTubeTranscriptApi
# from dotenv import load_dotenv
from pydantic import BaseModel
# load_dotenv()




# class TranscribeRequest(BaseModel):
#     video_url: str


def get_details_new(video_url):
    """
    Given YouTube Video URL as input, returns the transcription
    :param video_url:
    :return: <str> transcribed text, author name, title, video_id, description
    """
    youtube = YouTube(video_url)


    author = youtube.author

    title = youtube.streams[0].title



    transcript = ""
    video_id = extract.video_id(video_url)

    try:
      srt = YouTubeTranscriptApi.get_transcript(video_id)
      for line in srt:
        transcript = transcript + " " + line["text"]

    except:

      audio_stream = youtube.streams.filter(only_audio=True, file_extension="webm").last()
      audio_stream.download(filename='audio_temp.wav')
      audio_file =  open('audio_temp.wav', "rb")

      time.sleep(10)
      transcript = transcribe_audio(audio_file)
      audio_file.close()
      os.remove('audio_temp.wav')

    youtube.streams.first()
    description = youtube.description

    return [video_id, title, author, description, transcript]


In [83]:
example_url={
            "url1": "https://www.youtube.com/watch?v=flIG8Lw34Cw",
            "url2": "https://www.youtube.com/watch?v=ynfEvP5kYCk",
            "url3": "https://www.youtube.com/watch?v=28Jay-3S8fg",
            "url4": "https://www.youtube.com/watch?v=BnZU6qYVUl0",
            "url5": "https://www.youtube.com/watch?v=swst4yk-ow8",
            "url6": "https://www.youtube.com/watch?v=_0fwOm5K7Og"
            }

In [ ]:
i=1
for url in example_url.values():
  print(f"******* Example {i} ********")
  print(f"VIDEO_ID: {get_details_new(url)[0]}")
  print(f"TITLE: {get_details_new(url)[1]}")
  print(f"AUTHOR: {get_details_new(url)[2]}")
  print(f"DESCRIPTION: {get_details_new(url)[3]}")
  print(f"TRANSCRIPT: {get_details_new(url)[4]}")
  i+=1


******* Example 1 ********
VIDEO_ID: flIG8Lw34Cw
TITLE: D 30 Strategy | Make Daily 5000 to 10k risk Free
AUTHOR: Baap of Chart
DESCRIPTION: Today we will learn about the D30  Option Buying Sureshot Strategy which is one of the best option buying strategies for Intraday. it's based on my last 5 Years of experience and observations.

Course links: https://baapofchart.com/courses/
Charges: only 3300/- for old students and for new Students is 5100/- 
Payment mode upi: nasirboc@kotak
After payment email the payment screenshot to baapofchart@gmail.com, We will add you to the Support Group.

Everyone is just following strategy without any proper trading psychology that's why most people are losing money in trading.

I have given you the ultimate hack and how to follow that hack to recover all your trading losses.

So this video will help you to put the correct stop loss like a professional trader.

𝐂𝐨𝐧𝐧𝐞𝐜𝐭 𝐰𝐢𝐭𝐡 𝐮𝐬 𝐨𝐧 Official 𝐒𝐨𝐜𝐢𝐚𝐥 𝐌𝐞𝐝𝐢𝐚.

About me: https://bit.ly/2EhpXcr

Instagram: https:/